In [1]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 28.4 MB/s 
     |████████████████████████████████| 604 kB 81.1 MB/s 
     |████████████████████████████████| 120 kB 77.7 MB/s 
     |████████████████████████████████| 47.6 MB 1.2 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%cd data

/content/data


In [6]:
!7zr x /content/gdrive/MyDrive/data.7z

In [9]:
%cd /content

/content


In [ ]:
pip install 

In [10]:
!python3 create_lmdb_dataset.py --inputPath /content/data --gtFile /content/data/gt_train_2.txt --outputPath /conent/data_lmdb/train

Traceback (most recent call last):
  File "create_lmdb_dataset.py", line 3, in <module>
    import fire
ModuleNotFoundError: No module named 'fire'


In [ ]:
!python3 create_lmdb_dataset.py --inputPath /content/data --gtFile /content/data/gt_test_2.txt --outputPath /conent/data_lmdb/validation

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train.py --train_data data_lmdb/train --valid_data data_lmdb/validation --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction CTC --data_filtering_off --workers 0 --imgH 64 --imgW 200

In [ ]:
from easyocr.easyocr import *
import torch
from tqdm import tqdm

In [ ]:
def get_files(path):
    file_list = []

    files = [f for f in os.listdir(path) if not f.startswith('.')]  # skip hidden file
    files.sort()
    abspath = os.path.abspath(path)
    for file in files:
        file_path = os.path.join(abspath, file)
        file_list.append(file_path)

    return file_list, len(file_list)

In [ ]:
if __name__ == '__main__':
    reader = Reader(['ko'], gpu=True,
                    model_storage_directory="/content/gdrive/MyDrive/saved_models/TPS-ResNet-BiLSTM-CTC-Seed2022/model",
                    user_network_directory="/content/gdrive/MyDrive/saved_models/TPS-ResNet-BiLSTM-CTC-Seed2022/user_network",
                    recog_network='best_model')

    files, count = get_files('/content/gdrive/MyDrive/valid')
    files = sorted(files)

    result_list = []
    for idx, file in enumerate(tqdm(files)):
        filename = os.path.basename(file)

        result = reader.readtext(file, detail=0, paragraph=True)
        result = ''.join(result).strip().replace(' ', '')

        result_list.append(result)

100%|██████████| 3669/3669 [09:11<00:00,  6.65it/s]


In [ ]:
len(result_list)

3669

In [ ]:
result_list[26]

'하족객주'

In [ ]:
import pandas as pd
result_df = pd.read_csv('/content/gdrive/MyDrive/saved_models/TPS-ResNet-BiLSTM-CTC-Seed2022/sample_submission.csv')

In [ ]:
result_df['text'] = result_list

In [ ]:
result_df.to_csv('/content/gdrive/MyDrive/saved_models/TPS-ResNet-BiLSTM-CTC-Seed2022/final_submission_no_space.csv', index=False, encoding='utf-8-sig')